In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
file = open('./Data/WhatsApp Chat with Bse_Nse 🤑.txt', 'r', encoding='utf-8')

In [ ]:
data = file.read()

In [ ]:
print(data)

In [ ]:
import re

In [ ]:
pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{1,2}\s[a-z]{2}\s-\s'

In [ ]:
messages = re.split(pattern, data)[1:]
len(messages)

In [ ]:
date_time = re.findall(pattern, data)

In [ ]:
date_time = [x.replace('\u202f', ' ') for x in date_time]

In [ ]:
date_time

In [ ]:
len(date_time)

In [ ]:
df = pd.DataFrame({'USER_MESSAGES' : messages, 'DATE_TIME': date_time})

In [ ]:
df.head()

In [ ]:
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], format= '%d/%m/%y, %I:%M %p - ')

In [ ]:
df.head()

In [ ]:
df['YEAR'] = df.DATE_TIME.dt.year
df['MONTH'] = df.DATE_TIME.dt.month_name()
df['DAY'] = df.DATE_TIME.dt.day
df['HOUR'] = df.DATE_TIME.dt.hour
df['MINUTE'] = df.DATE_TIME.dt.minute

In [ ]:
df.head()

In [ ]:
df.USER_MESSAGES.head(20).to_list()

In [ ]:
df[['USERNAME', 'MESSAGE']] = df[df.USER_MESSAGES.str.contains(':')].USER_MESSAGES.str.split(':', 1, expand=True)
df['USERNAME'] = np.where(~df.USER_MESSAGES.str.contains(':'), 'Notification', df['USERNAME'])
df['MESSAGE'] = np.where(~df.USER_MESSAGES.str.contains(':'), df.USER_MESSAGES, df['MESSAGE'])

In [ ]:
df.head(50)

In [ ]:
df.MESSAGE.to_list()

In [ ]:
df.shape

In [ ]:
df.MESSAGE.str.split().str.len().sum()

In [ ]:
df[df.MESSAGE == ' <Media omitted>\n']

In [ ]:
cs = ' '.join(df.MESSAGE)

In [ ]:
# !pip install urlextract

In [ ]:
from urlextract import URLExtract

In [ ]:
extractor = URLExtract()
urls = extractor.find_urls(cs)

In [ ]:
len(urls)

In [ ]:
# Dictionary storing emoji counts 
from collections import defaultdict
emoji_count = defaultdict(int)
for i in df['MESSAGE']:
    for emoji in re.findall(u'[\U0001f300-\U0001f650]|[\u2000-\u3000]', i):
        emoji_count[emoji] += 1

print (emoji_count)

In [ ]:
pd.DataFrame({'Emoji' : emoji_count.keys(), 'Counts' : emoji_count.values()})

In [ ]:
dict(sorted(emoji_count.items(), key= lambda x: x[1], reverse=True)[:20])

In [ ]:
df.USERNAME.value_counts().sort_values(ascending=False).nlargest(10)

In [ ]:
pd.DataFrame({'No. of Messages' : df.USERNAME.value_counts().sort_values(ascending=False).nlargest(10)})

In [ ]:
import wordcloud

In [ ]:
str_messages = ''
str_messages += " ".join(df.MESSAGE) + " "

In [ ]:
str_messages_lower_case = str_messages.lower()

In [ ]:
wc = wordcloud.WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = wordcloud.STOPWORDS,
                min_font_size = 10).generate(str_messages_lower_case)

In [ ]:
plt.figure(figsize=(8,8), facecolor=None)
plt.imshow(wc)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
df.head()

In [ ]:
df["DAY_NAME"] = df.DATE_TIME.dt.day_name()

In [ ]:
df.head()

In [ ]:
day_wise_counts = df.groupby(['DAY_NAME'])['MESSAGE'].count()

In [ ]:
day_wise_counts = day_wise_counts.reset_index()
day_wise_counts['DAY_NAME'] = day_wise_counts['DAY_NAME'].apply(lambda x: x[:3])

In [ ]:
day_wise_counts.drop(['level_0', 'index'], axis=1, inplace=True)
day_wise_counts.set_index('DAY_NAME', inplace=True)

In [ ]:
day_wise_counts

In [ ]:
day_wise_counts.reindex(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

In [ ]:
day_wise_counts = df.groupby(['DAY_NAME'])['MESSAGE'].count().reset_index(drop=True).set_index(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

In [ ]:
day_wise_counts.values

In [ ]:
day_wise_counts.plot(kind='bar')

In [ ]:
import calendar

In [ ]:
df.MONTH.unique()

In [ ]:
df.groupby('MONTH')['MESSAGE'].count()

In [ ]:
temp = df[df.USERNAME == 'Sahil'].groupby('MONTH')['MESSAGE'].count()

In [ ]:
temp.reset_index()

In [ ]:
month_set = set()
for x in df.MONTH.unique():
    month_set.add(x[:3])
